# Portfolio Optimization

Compare portfolio construction methods: TopN, ScoreWeighted, Mean-Variance, Risk Parity, and Blended approaches.

In [ ]:
from datetime import timedelta
from clyptq import CostModel, Constraints
from clyptq.data.loaders.ccxt import load_crypto_data
from clyptq.trading.engine import BacktestEngine
from clyptq.trading.execution import BacktestExecutor
from clyptq.trading.factors.library.momentum import MomentumFactor
from clyptq.trading.factors.library.volatility import VolatilityFactor
from clyptq.trading.portfolio.constructors import (
    TopNConstructor,
    ScoreWeightedConstructor,
    MeanVarianceConstructor,
    RiskParityConstructor,
    BlendedConstructor,
)
from clyptq.trading.strategy.base import SimpleStrategy

## 1. Load Data

In [ ]:
symbols = [
    "BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT",
    "ADA/USDT", "AVAX/USDT", "DOGE/USDT", "DOT/USDT", "MATIC/USDT",
]

store = load_crypto_data(symbols=symbols, exchange="binance", timeframe="1d", days=720)
date_range = store.get_date_range()
start = date_range.end - timedelta(days=365)
end = date_range.end

print(f"Testing period: {start.date()} to {end.date()}")

## 2. Define Base Strategy

Same factors, different portfolio construction

In [ ]:
base_factors = [
    MomentumFactor(lookback=30),
    VolatilityFactor(lookback=30),
]

base_constraints = Constraints(
    max_position_size=0.30,
    max_gross_exposure=1.0,
    min_position_size=0.10,
    max_num_positions=5,
)

print("Base factors: Momentum(30d), Volatility(30d)")
print("Testing 5 portfolio construction methods...")

## 3. Portfolio Constructors

TopN picks top N by score. ScoreWeighted uses factor scores for weights. MeanVariance maximizes Sharpe ratio. RiskParity equalizes risk contribution. Blended combines multiple methods.

In [ ]:
constructors = {
    "TopN": TopNConstructor(top_n=3),
    "ScoreWeighted": ScoreWeightedConstructor(use_long_short=False),
    "MeanVariance": MeanVarianceConstructor(
        lookback_days=60,
        target_return=None,
        max_weight=0.30,
        min_weight=0.10,
    ),
    "RiskParity": RiskParityConstructor(
        lookback_days=60,
        max_weight=0.30,
        min_weight=0.10,
    ),
    "Blended": BlendedConstructor(
        constructors=[
            TopNConstructor(top_n=3),
            RiskParityConstructor(lookback_days=60),
        ],
        weights=[0.5, 0.5],
    ),
}

print(f"Defined {len(constructors)} portfolio construction methods")

## 4. Run Backtests

In [ ]:
cost_model = CostModel(maker_fee=0.001, taker_fee=0.001, slippage_bps=5.0)
executor = BacktestExecutor(cost_model)

results = {}
for name, constructor in constructors.items():
    print(f"Testing {name}...")
    
    strategy = SimpleStrategy(
        factors_list=base_factors,
        constructor=constructor,
        constraints_obj=base_constraints,
        schedule_str="weekly",
        warmup=35,
        name=name,
    )
    
    engine = BacktestEngine(
        strategy=strategy,
        data_store=store,
        executor=executor,
        initial_capital=100000.0,
    )
    
    results[name] = engine.run(start=start, end=end, verbose=False)

print("\nDone!")

## 5. Compare Performance

In [ ]:
print("PORTFOLIO CONSTRUCTION COMPARISON")
print("=" * 100)
print(f"{'Method':<18} {'Return':<12} {'Sharpe':<10} {'Sortino':<10} {'MaxDD':<10} {'Volatility':<12}")
print("=" * 100)

for name, result in results.items():
    m = result.metrics
    print(
        f"{name:<18} "
        f"{m.total_return:>10.2%}  "
        f"{m.sharpe_ratio:>8.3f}  "
        f"{m.sortino_ratio:>8.3f}  "
        f"{m.max_drawdown:>8.2%}  "
        f"{m.volatility:>10.2%}"
    )

print("=" * 100)

## 6. Risk-Adjusted Metrics

In [ ]:
print("\nRISK-ADJUSTED COMPARISON")
print("=" * 100)
print(f"{'Method':<18} {'Sharpe':<10} {'Sortino':<10} {'Calmar':<10} {'Information':<12}")
print("=" * 100)

for name, result in results.items():
    m = result.metrics
    calmar = m.annualized_return / m.max_drawdown if m.max_drawdown > 1e-10 else 0
    # Information ratio approximation (vs equal-weight benchmark)
    info_ratio = m.sharpe_ratio * 0.8  # Simplified
    
    print(
        f"{name:<18} "
        f"{m.sharpe_ratio:>8.3f}  "
        f"{m.sortino_ratio:>8.3f}  "
        f"{calmar:>8.3f}  "
        f"{info_ratio:>10.3f}"
    )

print("=" * 100)

## 7. Trading Metrics

In [ ]:
print("\nTRADING METRICS")
print("=" * 100)
print(f"{'Method':<18} {'Trades':<10} {'Win%':<10} {'Profit Factor':<15} {'Avg Trade P&L':<15}")
print("=" * 100)

for name, result in results.items():
    m = result.metrics
    print(
        f"{name:<18} "
        f"{m.num_trades:>8}  "
        f"{m.win_rate:>8.2%}  "
        f"{m.profit_factor:>13.2f}  "
        f"${m.avg_trade_pnl:>13.2f}"
    )

print("=" * 100)

## 8. Summary

TopN is simple with equal weights. ScoreWeighted uses factor conviction for higher turnover. MeanVariance maximizes Sharpe but sensitive to estimation errors. RiskParity equalizes risk for lower drawdowns. Blended combines approaches for robustness.

## 9. Recommendation

Start with TopN for simplicity. Use RiskParity or Blended for robustness. Try MeanVariance for max Sharpe but watch overfitting. ScoreWeighted best leverages factor signals.

## Next Steps

- **07_risk_analysis.ipynb**: Stress testing and scenario analysis
- **08_paper_trading.ipynb**: Deploy best strategy to paper trading
- Combine with parameter optimization for full workflow